---
# Ensemble learning

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

In [2]:
# creating the data

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)

In [3]:
# Creating the test and train dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
# Getting 4 different classifiers.

from sklearn.ensemble import RandomForestClassifier

In [5]:
from sklearn.ensemble import VotingClassifier

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.svm import SVC

Note: to be future-proof, we set solver="lbfgs", n_estimators=100, and gamma="scale" since these will be the default values in upcoming Scikit-Learn versions.

In [8]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)

In [9]:
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [10]:
svm_clf = SVC(gamma="scale", random_state=42)

In [11]:
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

In [12]:
# fitting the model

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                               

---
## Let’s look at each classifier’s accuracy on the test set:

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.912


---
Soft Voting

In [15]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                               

In [16]:
from sklearn.metrics import accuracy_score


In [17]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.92


# In case of soft voting the voting classifier have acquired more accuracy
---

---

# Bagging Classifier

In [18]:
# Bagging Classifier

from sklearn.ensemble import BaggingClassifier

In [19]:
from sklearn.tree import DecisionTreeClassifier

In [20]:
# Bagging Classifier with 500 decision trees and maximum of 100 instances each time

# bootstrap = True means sampling with replacement

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [21]:
# Importing the accuracy_score

from sklearn.metrics import accuracy_score

In [22]:
print(accuracy_score(y_test, y_pred))

0.904


---

### Creating a basic decision tree classifier


In [33]:
tree_clf = DecisionTreeClassifier(random_state=42)

In [34]:
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [35]:
y_pred_tree = tree_clf.predict(X_test)

In [36]:
print(accuracy_score(y_test, y_pred_tree))

0.856


---

## Plotting the difference between decision tree and bagging ensemble of 500 trees

In [38]:
from matplotlib.colors import ListedColormap

import matplotlib.pyplot as plt

import numpy as np

In [39]:
def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.45, -1, 1.5], alpha=0.5, contour=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", alpha=alpha)
    plt.axis(axes)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

In [40]:
fix, axes = plt.subplots(ncols=2, figsize=(10,4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(tree_clf, X, y)
plt.title("Decision Tree", fontsize=14)
plt.sca(axes[1])
plot_decision_boundary(bag_clf, X, y)
plt.title("Decision Trees with Bagging", fontsize=14)
plt.ylabel("")

plt.show()

---
# Out of Bag 

In [41]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)

In [42]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [43]:
bag_clf.oob_score_

0.9066666666666666

This oob classifier tells that this classifier is likely to perform 90.66% on the test dataset also. 

---

In [44]:
# Now, verify the classifier on the test data

from sklearn.metrics import accuracy_score

In [45]:
y_pred = bag_clf.predict(X_test)

In [46]:
accuracy_score(y_test, y_pred)

0.92

We get 92% accuracy which is ok close to the accuracy of our oob.

---

In [47]:
# oob_decision_function_ variable returns the class probabilities for the each training instances

bag_clf.oob_decision_function_

array([[0.41758242, 0.58241758],
       [0.3258427 , 0.6741573 ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.10309278, 0.89690722],
       [0.305     , 0.695     ],
       [0.02197802, 0.97802198],
       [0.98963731, 0.01036269],
       [0.97826087, 0.02173913],
       [0.79187817, 0.20812183],
       [0.00574713, 0.99425287],
       [0.74863388, 0.25136612],
       [0.81111111, 0.18888889],
       [0.95939086, 0.04060914],
       [0.06285714, 0.93714286],
       [0.        , 1.        ],
       [0.99470899, 0.00529101],
       [0.91194969, 0.08805031],
       [0.99462366, 0.00537634],
       [0.01098901, 0.98901099],
       [0.32105263, 0.67894737],
       [0.92513369, 0.07486631],
       [1.        , 0.        ],
       [0.9787234 , 0.0212766 ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.64673913, 0.35326087],
       [0.

For example, the oob evaluation estimates that the first training
instance has a 58.24% probability of belonging to the positive class (and 41.75% of
belonging to the negative class)

---

---

# Random Forest Implementation

In [49]:
# implementing the RandomForest with Decision Classifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16, random_state=42),
    n_estimators=500, max_samples=1.0, bootstrap=True, random_state=42)

In [50]:
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [51]:
print(y_pred)

[0 0 0 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0
 1 1 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 1
 0 0 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 1 1
 0 0 1 0 0 0 0 0 1 1 1 0 0 0]


In [52]:
# Now, implementing the randomforest with the help of RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier

In [53]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)

In [54]:
rnd_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=16, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [55]:
y_pred_rf = rnd_clf.predict(X_test)

In [56]:
print(y_pred_rf)

[0 0 0 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0
 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 1
 0 0 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 1 1
 0 0 1 1 0 0 0 0 1 1 1 0 0 0]


In [57]:
# checking if the predictions are identical or not

np.sum(y_pred == y_pred_rf) / len(y_pred)  # almost identical predictions

0.976

---

# Feature Importance

In [58]:
from sklearn.datasets import load_iris

In [59]:
iris = load_iris()

In [60]:
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)

In [61]:
rnd_clf.fit(iris["data"], iris["target"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [62]:
# getting the scores of each features

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.11249225099876375
sepal width (cm) 0.02311928828251033
petal length (cm) 0.4410304643639577
petal width (cm) 0.4233579963547682


In [63]:
rnd_clf.feature_importances_

array([0.11249225, 0.02311929, 0.44103046, 0.423358  ])

Seems like "Petal Length" and "petal Width" are important feature in comparison to the other two features.

---

---

## More Feature Importance

In [64]:
from sklearn.datasets import fetch_openml

In [65]:
mnist = fetch_openml('mnist_784', version=1)

In [66]:
mnist.target = mnist.target.astype(np.uint8)

In [67]:
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [68]:
rnd_clf.fit(mnist["data"], mnist["target"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

---

if you train a Random Forest classifier on the MNIST dataset (introduced in Chapter 3) and plot each pixel’s importance, you get the image represented in

In [69]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.hot,
               interpolation="nearest")
    plt.axis("off")

In [71]:
import matplotlib as mpl

In [72]:
plot_digit(rnd_clf.feature_importances_)

cbar = plt.colorbar(ticks=[rnd_clf.feature_importances_.min(), rnd_clf.feature_importances_.max()])
cbar.ax.set_yticklabels(['Not important', 'Very important'])

plt.show()

--- 

# Ada Boost Implementation

In [73]:
from sklearn.ensemble import AdaBoostClassifier

In [74]:
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)

In [75]:
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [76]:
plot_decision_boundary(ada_clf, X, y)

---

# Gradient Boost

In [77]:
np.random.seed(42)

In [78]:
X = np.random.rand(100, 1) - 0.5

In [79]:
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [80]:
from sklearn.tree import DecisionTreeRegressor

In [81]:
tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)

In [82]:
tree_reg1.fit(X, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [83]:
# Error from the result

y2 = y - tree_reg1.predict(X)

In [84]:
# Training on the basis of previous Error

tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)

In [85]:
tree_reg2.fit(X, y2)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [86]:
# Again Training on the basis of previous Error

y3 = y2 - tree_reg2.predict(X)

In [87]:
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)

In [88]:
tree_reg3.fit(X, y3)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [89]:
X_new = np.array([[0.8]])

In [90]:
# Summing the predictions

y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [91]:
# This is the prediction of X_new input

y_pred

array([0.75026781])